## Npz files for Visbrain for all subjects

In [1]:
from brainpipe.system import study
import numpy as np
from os import listdir,makedirs
from os.path import isfile, join,exists
import numpy as np

st = study('Olfacto')
step = 'odor'
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ'] #respectively S0 S1 S2 S3 S4 S5
freqs = ['0_VLFC', '1_delta', '2_theta', '3_alpha']
conds = {'0':['good','bad','GoodBad'],'1':['poor','partial','EpiPerf'],'2':['partial','detailed','EpiPerf'],
         '3':['poor','detailed','EpiPerf']}

for cond in conds:
    path_classif = join(st.path, 'classified/1_Classif_Phase_'+conds[cond][0].capitalize()+'_'+conds[cond][1].capitalize()+'_EpiPerf_4500_expi_noart/')
    path_data = join(st.path, 'feature/2_Phase_Encoding_'+conds[cond][2]+'_4500_expi_noart/')
    path2save = join(st.path, 'figures_npz/1_'+conds[cond][0]+'_'+conds[cond][1]+'_4500_expi_noart/')

    ## Create a npz file with all sources informations + POWER (subjects, coords, labels)
    for j,freq in enumerate(freqs):
        su_codes, s_xyz, s_channels,s_labels, s_elec, s_elec_p0, s_elec_p1 = np.array([]),np.array([]), np.array([]), np.array([]),np.array([]),np.array([]),np.array([])
        s_stats, s_stats_t0, s_da, s_th_05, s_th_01 = np.array([]),np.array([]),np.array([]),np.array([]),np.array([])
        s_BA, s_AAL, s_AAL_RL, s_elec_p0_std, s_elec_p1_std = np.array([]), np.array([]),np.array([]),np.array([]),np.array([])
        print(freq)
        for i,su in enumerate(subjects):
            print(su)
            filename0 = path_data+su+'_odor_'+conds[cond][0]+'_bipo_sel_phys_phase.npz'
            filename1 = path_data+su+'_odor_'+conds[cond][1]+'_bipo_sel_phys_phase.npz'
            if isfile(filename0) == True & isfile(filename1) == True:
                p0, p1 = np.load(filename0), np.load(filename1)
                print('files in phase', p0.files,p0['sPhase'].shape)
                xyz_all = p0['xyz']
                s_time = p0['time']
                #print('time vector', p0['time'])
                for elec in range(xyz_all.shape[0]):
                    ### INFO electrodes for all subjects
                    su_codes = np.vstack((su_codes,'S'+str(i))) if np.size(su_codes) else 'S'+str(i)
                    xyz = xyz_all[elec]
                    s_xyz = np.vstack((s_xyz, xyz)) if np.size(s_xyz) else xyz
                    aal = p0['s_aal'][elec]
                    s_AAL = np.vstack((s_AAL, aal)) if np.size(s_AAL) else aal
                    aal_RL = aal[:-4]
                    s_AAL_RL = np.vstack((s_AAL_RL, aal_RL)) if np.size(s_AAL_RL) else aal_RL
                    BA = p0['s_BA'][elec]
                    s_BA = np.vstack((s_BA, BA)) if np.size(s_BA) else BA
                    label = p0['labels'][elec]
                    #label_num = label+'_'+str(elec)
                    channel = p0['channels'][elec]               
                    s_labels = np.vstack((s_labels, label)) if np.size(s_labels) else label
                    s_channels = np.vstack((s_channels, channel)) if np.size(s_channels) else channel
                    ### PHASE values for all electrodes for 1 freq for Good & Bad conditions
                    elec_p0 = p0['sPhase'][elec,j,:39,:]
                    elec_p1 = p1['sPhase'][elec,j,:39,:]
                    #print(elec_pow0.shape, elec_pow1.shape)
                    ## AVERAGE POWER power across trials + select values only AFTER odor perception (14 windows)
                    elec_p0_mean = np.mean(elec_p0, axis=1)
                    elec_p1_mean = np.mean(elec_p1, axis=1)
                    s_elec_p0 = np.vstack((s_elec_p0, elec_p0_mean)) if np.size(s_elec_p0) else elec_p0_mean
                    s_elec_p1 = np.vstack((s_elec_p1, elec_p1_mean)) if np.size(s_elec_p1) else elec_p1_mean
                    elec_p0_std = np.std(elec_p0, axis=1)
                    elec_p1_std = np.std(elec_p1, axis=1)
                    s_elec_p0_std = np.vstack((s_elec_p0_std, elec_p0_std)) if np.size(s_elec_p0_std) else elec_p0_std
                    s_elec_p1_std = np.vstack((s_elec_p1_std, elec_p1_std)) if np.size(s_elec_p1_std) else elec_p1_std
                    s_elec = np.vstack((s_elec,str(elec))) if np.size(s_elec) else str(elec)
                    ### STATS & DECODING ACCURACY for all electrodes
                    da = np.mean(np.load(join(path_classif, freq+'/auc/'+su+'_auc_'+conds[cond][0].capitalize()+'_'+conds[cond][1].capitalize()+'_'+label+'_('+str(elec)+').npy')),axis=0)
                    th_05 = np.load(join(path_classif, freq+'/auc/'+su+'_th_0_05_perm_'+label+'_('+str(elec)+').npy'))
                    th_01 = np.load(join(path_classif, freq+'/auc/'+su+'_th_0_01_perm_'+label+'_('+str(elec)+').npy'))
                    #print('for one elec', stats.shape, da.shape,th_05.shape, th_01.shape)
                    ### AVERAGE stats across repetitions for da + select only values AFTER odor perception
                    s_da = np.vstack((s_da, da)) if np.size(s_da) else da
                    s_th_05 = np.vstack((s_th_05, th_05)) if np.size(s_th_05) else th_05
                    s_th_01 = np.vstack((s_th_01, th_01)) if np.size(s_th_01) else th_01
            else:
                print('this patient does not have both conditions')
        print ('before', 'p1',s_elec_p1.shape, 'p0',s_elec_p0.shape,'std0', s_elec_p0_std.shape,
               'da', s_da.shape, 'th',s_th_05.shape, s_th_01.shape,'aal',s_AAL.shape,'BA',s_BA.shape)   
        su_codes = np.squeeze(su_codes)
        s_labels = np.squeeze(s_labels)
        s_AAL = np.squeeze(s_AAL)
        s_AAL_RL = np.squeeze(s_AAL_RL)
        s_BA = np.squeeze(s_BA)
        s_channels = np.squeeze(s_channels)
        s_elec = np.squeeze(s_elec)
        s_da = np.squeeze(s_da)
        s_th_05 = np.squeeze(s_th_05)
        s_th_01 = np.squeeze(s_th_01)
        print ('after', 'p1',s_elec_p1.shape, 'p0',s_elec_p0.shape,
               'da', s_da.shape, 'th',s_th_05.shape, s_th_01.shape,'aal',s_AAL.shape, 'BA',s_BA.shape)
        su_coord_label = {'su_codes':su_codes, 's_xyz':s_xyz, 's_labels':s_labels,'s_channels':s_channels, 
                          's_elec':s_elec, 's_elec_p0':s_elec_p0,'s_time':s_time,
                         's_elec_p1':s_elec_p1,'s_da':s_da,'s_th_05':s_th_05, 
                          's_th_01':s_th_01,'s_aal':s_AAL, 's_BA':s_BA, 's_aal_RL':s_AAL_RL,
                         's_elec_p0_std':s_elec_p0_std, 's_elec_p1_std':s_elec_p1_std}
        file_source = 'All_subjects_sources_phase_'+freq+'_'+step+'_'+conds[cond][0]+'_'+conds[cond][1]+'_Expi_sel_phys.npz'
        if not exists(path2save):
            makedirs(path2save)
        np.savez(path2save+file_source,**su_coord_label)

-> Olfacto loaded
0_VLFC
LEFC
files in phase ['fname', 'phase_pval', 'xPhase', 'sPhase', 'xyz', 's_BA', 'time', 'labels', 'f', 'channels', 's_aal'] (148, 4, 39, 23)
time vector [ 0.34960938  0.44921875  0.54882812  0.6484375   0.74804688  0.84765625
  0.94726562  1.046875    1.14648438  1.24609375  1.34570312  1.4453125
  1.54492188  1.64453125  1.74414062  1.84375     1.94335938  2.04296875
  2.14257812  2.2421875   2.34179688  2.44140625  2.54101562  2.640625
  2.74023438  2.83984375  2.93945312  3.0390625   3.13867188  3.23828125
  3.33789062  3.4375      3.53710938  3.63671875  3.73632812  3.8359375
  3.93554688  4.03515625  4.13476562]
CHAF
files in phase ['fname', 'phase_pval', 'xPhase', 'sPhase', 'xyz', 's_BA', 'time', 'labels', 'f', 'channels', 's_aal'] (93, 4, 39, 16)
time vector [ 0.34960938  0.44921875  0.54882812  0.6484375   0.74804688  0.84765625
  0.94726562  1.046875    1.14648438  1.24609375  1.34570312  1.4453125
  1.54492188  1.64453125  1.74414062  1.84375     1.943